In [1]:
lines = []
with open('data/all_text.txt', 'r', encoding='utf-8') as file:
    for line in file:
        lines.append(line.strip())

In [2]:
import re
def remove_urls(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # 使用 re.sub() 函数替换所有匹配的 URL 为 ""
    text_without_urls = re.sub(url_pattern, '', text)
    specific_text_pattern = re.compile(r'扫描下方二维码关注公众号|提取码|关注|科学上网|回复关键词|侵权|版权|致谢|引用|LICENSE'
                                   r'|组队打卡|任务打卡|组队学习的那些事|学习周期|开源内容|打卡|组队学习|链接')
    text_without_urls = re.sub(specific_text_pattern, '', text)
    return text_without_urls
from langchain.schema import Document
docs = []
for idx, line in enumerate(lines):
    line = remove_urls(line.strip("\n").strip())
    words = line.split("\t")
    docs.append(Document(page_content=words[0], metadata={"id": idx, "source": "all_text.txt"}))

In [ ]:
docs

In [4]:
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

top_k = 20
import torch

# 检查 CUDA 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = '/mnt/workspace/.cache/modelscope/hub/maidalun/bce-embedding-base_v1'
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:55: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [ ]:
vectorstore = Chroma.from_documents(documents=docs[:], embedding=hf, persist_directory="db_car_chroma_bce")

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": top_k}
)
question = "怎样加热座椅？"
retri_re = retriever.invoke(question)

In [ ]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs, hf)

In [ ]:
db.save_local("db_car_faiss_bce")

new_db = FAISS.load_local("faiss_index", hf)

docs = new_db.similarity_search("怎样加热座椅？")